In [1]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from xgboost import XGBClassifier
import pandas as pd
from sklearn.metrics import roc_auc_score, classification_report
import json

In [2]:
train = pd.read_csv("../data/interim/train.csv")
test = pd.read_csv("../data/interim/test.csv")
train.head()

,peserta.STD(fkrtl.lama_hari_kunjungan),peserta.SUM(fkrtl.jml_kunjungan_fkrtl),jml_kunjungan_fkrtl,status_pulang_peserta,MONTH(tanggal_pulang),YEAR(tanggal_pulang),peserta.SUM(fkrtl.lama_hari_kunjungan),peserta.status_peserta,peserta.SUM(fktp.jarak_hari_antar_kunjungan),MONTH(tanggal_datang),...,peserta.MAX(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fkrtl.no_keluarga),peserta.MIN(fkrtl.spesifikasi_kelompok_kasus),peserta.SUM(fkrtl.bobot),peserta.STD(fkrtl.biaya_tagih),peserta.SUM(fktp.bobot),peserta.MIN(fkrtl.biaya_tagih),peserta.gender,peserta.COUNT(fktp),readmitted_30d
0,0.227699,-0.094196,0.864951,0,4,2020,0.268705,2,-1.721583,4,...,0.000000,-0.356290,-0.504986,0.542504,-0.589181,-0.762970,-1.170089,1,-0.876887,0
1,-0.298560,-0.225341,-0.489105,3,0,2020,-0.192478,2,-1.721583,0,...,0.000000,-0.346450,-0.823839,-0.005455,-0.906346,-0.671221,-0.605438,1,-0.784533,1
2,0.020066,-0.094196,0.187923,3,7,2020,0.360941,1,0.433170,7,...,-0.478321,-0.172754,-0.823839,0.495776,-0.355748,1.089862,-0.454042,0,0.970200,0
3,-0.422445,1.086116,2.896035,3,11,2020,1.652253,1,0.860893,11,...,-0.001438,-0.091115,-0.504986,2.748588,-0.092010,0.541366,-1.153514,1,0.323719,1
4,-0.313880,4.823769,3.573063,3,3,2020,1.560016,2,-1.140526,3,...,-0.250247,0.746741,-0.823839,4.267781,-0.224923,-0.610887,-0.894251,1,-0.692178,1


In [3]:
X_train = train.drop(columns=['readmitted_30d'])
y_train = train[['readmitted_30d']]
X_test = test.drop(columns=['readmitted_30d'])
y_test = test[['readmitted_30d']]

In [4]:
y_train.value_counts()

readmitted_30d
0                 18735
1                  3526
Name: count, dtype: int64

In [5]:
with open("../artifacts/best_params.json","r") as f:
    best_params = json.load(f)
    
pipeline = ImbPipeline([
    ('smote', SMOTE(sampling_strategy=0.5, random_state=42)),
    ('model', XGBClassifier(**best_params))
])

pipeline.fit(X_train, y_train)

,steps,"[('smote', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,sampling_strategy,0.5
,random_state,42
,k_neighbors,5
,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None


In [6]:
# Evaluate
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

preds = pipeline.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, preds)
print("ROC AUC Score with SMOTE:", auc)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      4716
           1       0.62      0.63      0.62       850

    accuracy                           0.88      5566
   macro avg       0.77      0.78      0.78      5566
weighted avg       0.89      0.88      0.88      5566

ROC AUC Score with SMOTE: 0.9359946115850921
